<a href="https://colab.research.google.com/github/TusharPaul01/Work/blob/main/kusho_GetAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests


In [ ]:
!pip install google-generativeai langchain-google-genai streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 14.5 MB/s eta 0:00:00


In [ ]:
import os
import google.generativeai as genai

os.environ['GOOGLE_API_KEY'] = "*******************"
genai.configure(api_key = os.environ['GOOGLE_API_KEY'])

In [ ]:
from IPython.display import Markdown
def geturl(company_name):
  model = genai.GenerativeModel('gemini-pro')
  response = model.generate_content(f"API key explorer url of {company_name} company")

  # Assuming response.text contains the URL
  link_text = "Link Text"
  url = response.text

  # Create the Markdown link
  markdown_link = f"[{link_text}]({url})"
  display(Markdown(markdown_link))

  # Show the URL as well
  print(f"URL: {url}")
  return url


In [ ]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin, urlparse

def get_links(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = [a['href'] for a in soup.find_all('a', href=True)]
        return links
    except Exception as e:
        print(f"Error fetching links from {url}: {e}")
        return []

def crawl_website(start_url, max_depth=3):
    visited_urls = set()
    postman_collection_links = []
    api_docs_links = []
    openapi_spec_links = []

    def crawl(url, depth):
        if depth > max_depth or url in visited_urls:
            return
        visited_urls.add(url)

        print(f"Crawling {url} at depth {depth}")

        links = get_links(url)

        for link in links:
            absolute_url = urljoin(url, link)
            if urlparse(absolute_url).scheme in ('http', 'https') and urlparse(absolute_url).netloc:
                if 'postman' in absolute_url.lower():
                    postman_collection_links.append(absolute_url)
                elif 'apidocs' in absolute_url.lower():
                    api_docs_links.append(absolute_url)
                elif 'openapi' in absolute_url.lower():
                    openapi_spec_links.append(absolute_url)

                crawl(absolute_url, depth + 1)

    crawl(start_url, 1)

    return postman_collection_links, api_docs_links, openapi_spec_links

def find_resources(company_name):
    # Example starting point for crawling  https://api-explorer.bqecore.com/#About-Core-APIs
    #start_url = f'https://www.example.com/{company_name}'
    #start_url = f'https://www.{company_name}.com'
    #start_url = f'https://api-explorer.{company_name}.com/'
    #start_url = f'https://api-explorer.{company_name.lower()}.com/'
    start_url = geturl(company_name)
    # Set the maximum depth for crawling
    max_depth = 1

    # Call the web crawling function
    postman_collection_links, api_docs_links, openapi_spec_links = crawl_website(start_url, max_depth)

    return postman_collection_links, api_docs_links, openapi_spec_links

company_name = input("Enter company name: ")
postman_collection_links, api_docs_links, openapi_spec_links = find_resources(company_name)

if postman_collection_links:
    print("Found Postman Collection Links:")
    for link in postman_collection_links:
        print(link)

if api_docs_links:
    print("\nFound API Docs Links:")
    for link in api_docs_links:
        print(link)

if openapi_spec_links:
    print("\nFound OpenAPI Spec Links:")
    for link in openapi_spec_links:
        print(link)

if not postman_collection_links and not api_docs_links and not openapi_spec_links:
    print("\nNo relevant resources found.")


Enter company name: Instagram


[Link Text](https://developers.facebook.com/tools/explorer/1918795826925504/)

URL: https://developers.facebook.com/tools/explorer/1918795826925504/
Crawling https://developers.facebook.com/tools/explorer/1918795826925504/ at depth 1

No relevant resources found.
